## [DACON] 진동데이터 활용 충돌체 탐지 AI 경진대회
## 두부(팀명) (Team name)
## 2020년 7월 17일 (제출날짜) (Submission date)

중간 중간에 코드가 미숙한 부분들이 많이 있습니다. 양해해주시면 감사하겠습니다.

## 1. 라이브러리 및 데이터
## Library & Data

In [112]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import random
random.seed(777)
np.random.seed(1)

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from tqdm import tqdm
import lightgbm as lgbm
import re
from sklearn.ensemble import RandomForestRegressor 
os.getcwd()
os.chdir('C:\\Users\\ParkGiChan\\Desktop\\DataAnalysis\\DACON_KAERI')

In [113]:
import os
import json
import numpy as np
from tqdm import tqdm
import jovian
%matplotlib inline
import matplotlib.pyplot as plt

import keras
import tensorflow as tf

from keras.layers import Dense, Activation, Conv2D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D
import keras.backend as K
from keras.regularizers import l1_l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback, EarlyStopping
from sklearn.datasets.samples_generator import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential, load_model

# Check package versions

In [114]:
print("keras_version : " + str(keras.__version__))
print("numpy_version : " + str(np.__version__))
print("pandas_version : " + str(pd.__version__))
print("tensorflow_version :" + str(tf.__version__))

keras_version : 2.1.5
numpy_version : 1.16.6
pandas_version : 0.25.3
tensorflow_version :1.8.0


## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

## 1. Case1, Case2 (변수추가 + Time 200까지만) 
Case1 : Cosine scheduler 사용 \
Case2 : X \
-> 데이터가 적고, metric이 너무 예민해서 그런지, scheduler 혹은 lr에  따라 학습 되는 것이 너무 달라 앙상블 해줬습니다.\
-> LB : 0.0035

In [115]:
train = pd.read_csv("train_features.csv", index_col=0)
test = pd.read_csv("test_features.csv", index_col=0)
y_train = pd.read_csv("train_target.csv", index_col=0)
y_test = pd.read_csv("sample_submission.csv", index_col=0)

In [116]:
train = train.loc[train['Time'] <= 0.000004*200]
test = test.loc[test['Time'] <= 0.000004*200]

In [117]:
def transform1(train, n1, n2):
    train_time = train.copy()
    train_time['S1'] = [1 if i != 0 else 0 for i in train['S1']]
    train_time['S2'] = [1 if i != 0 else 0 for i in train['S2']]
    train_time['S3'] = [1 if i != 0 else 0 for i in train['S3']]
    train_time['S4'] = [1 if i != 0 else 0 for i in train['S4']]
    train_time.drop(['Time'], axis=1, inplace=True)
    
    train_time = train_time.sum(axis=1)
    train_time = train_time.transform(lambda x: (x-x.mean())/(x.std()))
    train_time = np.array(train_time).reshape(n1, n2, 1, 1)
    
    return train_time

train_time = transform1(train, 2800, 200)
test_time = transform1(test, 700, 200)

train = train.transform(lambda x: (x-x.mean())/(x.std()))
test = test.transform(lambda x: (x-x.mean())/(x.std()))

###

X_data = train.iloc[:,:]
X_data = np.array(X_data).reshape((2800,200,5,1))

X_data_test = test.iloc[:,:]
X_data_test = np.array(X_data_test).reshape((700,200,5,1))

Y_data = np.array(y_train.copy())

### Merge

X_data = np.concatenate([X_data, train_time], axis=2)
X_data_test = np.concatenate([X_data_test, test_time], axis=2)

In [118]:
X_data_Case12 = X_data.copy()
X_data_test_Case12 = X_data_test.copy()
Y_data_Case12 = Y_data.copy()

## 3.  Case3 (kernel regularizer 사용)
앙상블을 하면 어떨까 해서 만들어 봤습니다. \
변수만추가하고, kerenl regularizer을 사용하였습니다.

In [119]:
train = pd.read_csv("train_features.csv", index_col=0)
test = pd.read_csv("test_features.csv", index_col=0)
y_train = pd.read_csv("train_target.csv", index_col=0)
y_test = pd.read_csv("sample_submission.csv", index_col=0)

In [120]:
train_time = transform1(train, 2800, 375)
test_time = transform1(test, 700, 375)

train = train.transform(lambda x: (x-x.mean())/(x.std()))
test = test.transform(lambda x: (x-x.mean())/(x.std()))

###

X_data = train.iloc[:,:]
X_data = np.array(X_data).reshape((2800,375,5,1))

X_data_test = test.iloc[:,:]
X_data_test = np.array(X_data_test).reshape((700,375,5,1))

Y_data = np.array(y_train.copy())

### Merge

X_data = np.concatenate([X_data, train_time], axis=2)
X_data_test = np.concatenate([X_data_test, test_time], axis=2)

In [121]:
X_data_Case3 = X_data.copy()
X_data_test_Case3 = X_data_test.copy()
Y_data_Case3 = Y_data.copy()

## 4.  Case4
baseline model입니다. 유용균박사님 코드에서 cosine scheduler만 추가한 것입니다.\
이 또한 앙상블을 하기 위해 만들어 봤습니다

In [122]:
train = pd.read_csv("train_features.csv", index_col=0)
test = pd.read_csv("test_features.csv", index_col=0)
y_train = pd.read_csv("train_target.csv", index_col=0)
y_test = pd.read_csv("sample_submission.csv", index_col=0)

In [123]:
train = train.transform(lambda x: (x-x.mean())/(x.std()))
test = test.transform(lambda x: (x-x.mean())/(x.std()))

###

X_data = train.iloc[:,:]
X_data = np.array(X_data).reshape((2800,375,5,1))

X_data_test = test.iloc[:,:]
X_data_test = np.array(X_data_test).reshape((700,375,5,1))

Y_data = np.array(y_train.copy())

In [124]:
X_data_Case4 = X_data.copy()
X_data_test_Case4 = X_data_test.copy()
Y_data_Case4 = Y_data.copy()

## check

In [125]:
print("Case12 nrow: " + str(X_data_Case12.shape[1]) + " / Case12 ncol : " + str(X_data_Case12.shape[2]))
print("Case3 nrow : " + str(X_data_Case3.shape[1]) + " / Case3 ncol : " + str(X_data_Case3.shape[2]))
print("Case4 nrow : " + str(X_data_Case4.shape[1]) + " / Case4 ncol : " + str(X_data_Case4.shape[2]))

Case12 nrow: 200 / Case12 ncol : 6
Case3 nrow : 375 / Case3 ncol : 6
Case4 nrow : 375 / Case4 ncol : 5


In [126]:
print("Case12 nrow: " + str(X_data_test_Case12.shape[1]) + " / Case12 ncol : " + str(X_data_test_Case12.shape[2]))
print("Case3 nrow : " + str(X_data_test_Case3.shape[1]) + " / Case3 ncol : " + str(X_data_test_Case3.shape[2]))
print("Case4 nrow : " + str(X_data_test_Case4.shape[1]) + " / Case4 ncol : " + str(X_data_test_Case4.shape[2]))

Case12 nrow: 200 / Case12 ncol : 6
Case3 nrow : 375 / Case3 ncol : 6
Case4 nrow : 375 / Case4 ncol : 5


## 3. 탐색적 자료분석
## Exploratory Data Analysis


## 4. 변수 선택 및 모델 구축
기본적으로 유용균 박사님 코드에서 5fold + cosine scheduler 등등만 추가한 것입니다

In [127]:
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(777)
rn.seed(777)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from numpy.random import seed
seed(777)
from tensorflow import set_random_seed
set_random_seed(777)

In [128]:
import math
from keras.callbacks import Callback
class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        #if self.verbose &gt; 0:
        #    print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
        #          'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [129]:
import numpy as np

def kaeri_metric(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: KAERI metric
    '''
    
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)


### E1과 E2는 아래에 정의됨 ###

def E1(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: distance error normalized with 2e+04
    '''
    
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)


def E2(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    
    
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))

In [130]:
weight1 = np.array([1,1,0,0])
weight2 = np.array([0,0,1,1])

def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult))


def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04

def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult)*weight2)

In [131]:
def plot_error(type_id,pred,true):
    print(pred.shape)

    if type_id == 0:
        _name = 'x_pos'
    elif type_id == 1:
        _name = 'y_pos'
    elif type_id == 2:
        _name = 'mass'
    elif type_id == 3:
        _name = 'velocity'
    elif type_id == 4:
        _name = "distance"
    else:
        _name = 'error'

    x_coord = np.arange(1,pred.shape[0]+1,1)
    if type_id < 2:
        Err_m = (pred[:,type_id] - true[:,type_id])
    elif type_id < 4:
        Err_m = ((pred[:,type_id] - true[:,type_id])/true[:,type_id])*100
    else:
        Err_m = ((pred[:,0]-true[:,0])**2+(pred[:,1]-true[:,1])**2)**0.5


    fig = plt.figure(figsize=(8,6))
    # plt.rcParams["font.family"]="Times New Roman"
    plt.rcParams["font.size"]=15
    plt.scatter(x_coord, Err_m, marker='o')
    plt.title("%s Prediction for Training Data" % _name, size=20)
    plt.xlabel("Data ID", labelpad=10, size=20)
    plt.ylabel("Prediction Error of %s," % _name, labelpad=10, size=20)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.ylim(-100., 100.)
    plt.xlim(0, pred.shape[0]+1)

    plt.show()
    
    print(np.std(Err_m))
    print(np.max(Err_m))
    print(np.min(Err_m))
    return Err_m

In [132]:
def set_model(train_target, n_sam,n_col, regularizer_True):  # 0:x,y, 1:m, 2:v
    
    activation = 'elu'
    padding = 'same'
    model = Sequential()
    nf = 16
    fs = (3,1)
    
    model.add(Conv2D(nf,fs, padding=padding, activation=activation,input_shape=(n_sam,n_col,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*2,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*4,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*8,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*16,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*32,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    model.add(Flatten())
    model.add(Dense(128, activation ='elu'))
    model.add(Dense(64, activation ='elu'))
    model.add(Dense(32, activation ='elu'))
    model.add(Dense(16, activation ='elu'))
    
    if regularizer_True:
        model.add(Dense(4, kernel_regularizer=l1_l2(l1=0.001)))
    else:
        model.add(Dense(4))
    

    optimizer = keras.optimizers.Adam()
    #optimizer = keras.optimizers.RMSprop()
    
    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    else: # only for V
        weight2 = np.array([0,0,0,1])
       
    if train_target==0:
        model.compile(loss=my_loss_E1,
                  optimizer=optimizer,
                 )
    else:
        model.compile(loss=my_loss_E2,
                  optimizer=optimizer,
                 )
       
    model.summary()

    return model

In [133]:
def train_CNN(model,X,Y,X_val,Y_val, save_point, enum, train_target, scheduler_True):
    
    from numpy.random import seed
    seed(777)
    from tensorflow import set_random_seed
    set_random_seed(777)
    
    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_loss:.4f}.hdf5'
    best_save = ModelCheckpoint('best_m_' + str(save_point) + "_" + str(enum) + "_" + str(train_target) + '.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    cosine_scheduler = CosineAnnealingScheduler(T_max=100, eta_max=6e-3, eta_min=1e-6)

    early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
    
    my_seed = 777
    np.random.seed(my_seed)
    if scheduler_True:
        history = model.fit(X, Y,
                      epochs=10000,
                      batch_size=128,
                      shuffle=True,
                      validation_data=[X_val, Y_val],
                      verbose = 2,
                      callbacks=[best_save, early_stop,cosine_scheduler],
                      )
    else:
        history = model.fit(X, Y,
                      epochs=10000,
                      batch_size=128,
                      shuffle=True,
                      validation_data=[X_val, Y_val],
                      verbose = 2,
                      callbacks=[best_save, early_stop],
                      )
    

    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    plt.show()    
    
    return model

In [134]:
def load_best_model(train_target, save_point, enum):

    if train_target == 0:
        model = load_model('best_m_' + str(save_point) + "_" + str(enum) + "_" + str(train_target) + '.hdf5' , custom_objects={'my_loss_E1': my_loss, })
    else:
        model = load_model('best_m_' + str(save_point) + "_" + str(enum) + "_" + str(train_target) + '.hdf5' , custom_objects={'my_loss_E2': my_loss, })

    score = model.evaluate(X_data, Y_data, verbose=0)
    print('loss:', score)

    pred = model.predict(X_data)

    i=0

    print('정답(original):', Y_data[i])
    print('예측값(original):', pred[i])

    print(E1(pred, Y_data))
    print(E2(pred, Y_data))
    #print(E2M(pred, Y_data))
    #print(E2V(pred, Y_data))    
    
    if train_target ==0:
        plot_error(4,pred,Y_data)
    elif train_target ==1:
        plot_error(2,pred,Y_data)
    elif train_target ==2:
        plot_error(3,pred,Y_data)    
    
    return model

In [135]:
lr = 3e-4
lr_d = 0.0
patience = 30
dr_rate = 0

def run_model(X_data, X_data_test, Y_data, save_point,scheduler_True,regularizer_True):
    submit = pd.read_csv('sample_submission.csv', index_col=0)
    submit.iloc[:,:] = 0

    lr = 3e-4
    lr_d = 0.0
    patience = 30
    dr_rate = 0
    
    n_col = X_data.shape[2]
    n_sam = X_data.shape[1]
    
    for enum, (train_index,valid_index) in enumerate(group_kfold.split(X_data,Y_data,groups)):
        X_train = X_data[train_index]
        Y_train = Y_data[train_index]
    
        X_val = X_data[valid_index]
        Y_val = Y_data[valid_index]
    
    
        for train_target in range(3):
            model = set_model(train_target, n_sam,n_col, regularizer_True)
        
            train_CNN(model,X_train, Y_train, X_val, Y_val, save_point, enum, train_target,scheduler_True)    
            best_model = load_best_model(train_target, save_point, enum)

   
            pred_data_test = best_model.predict(X_data_test)
            pred_data_valid = best_model.predict(X_val)
    
            if train_target == 0: # x,y 학습
                submit.iloc[:,0] += pred_data_test[:,0]/5
                submit.iloc[:,1] += pred_data_test[:,1]/5
                
            elif train_target == 1: # m 학습
                submit.iloc[:,2] += pred_data_test[:,2]/5
        
            elif train_target == 2: # v 학습
                submit.iloc[:,3] += pred_data_test[:,3]/5    
    
    return submit

## GroupKFold

In [136]:
y_train = pd.read_csv("train_target.csv", index_col=0)

In [137]:
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.model_selection import GroupKFold
group_kfold = GroupKFold(n_splits=5)
groups = y_train['X'].astype('str') + "_" + y_train['Y'].astype('str') + \
    "_" + y_train['M'].astype('str') + "_" + y_train['V'].astype('str')
group_kfold.get_n_splits(X_data, Y_data, groups)

5

## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

# CNN
keras의 경우 seed 고정을 해도, 값이 정확히 같게 나오지 않아, 같은 모델들을 3번씩 반복했습니다.

# Case1

In [138]:
X_data = X_data_Case12.copy()
X_data_test = X_data_test_Case12.copy()
Y_data = Y_data_Case12.copy()

In [139]:
submit_case1_1 = run_model(X_data,X_data_test, Y_data, 1,True,False)
submit_case1_2 = run_model(X_data,X_data_test, Y_data, 2,True,False)
submit_case1_3 = run_model(X_data,X_data_test, Y_data, 3,True,False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_181 (Conv2D)          (None, 200, 6, 16)        64        
_________________________________________________________________
batch_normalization_181 (Bat (None, 200, 6, 16)        64        
_________________________________________________________________
max_pooling2d_181 (MaxPoolin (None, 100, 6, 16)        0         
_________________________________________________________________
conv2d_182 (Conv2D)          (None, 100, 6, 32)        1568      
_________________________________________________________________
batch_normalization_182 (Bat (None, 100, 6, 32)        128       
_________________________________________________________________
max_pooling2d_182 (MaxPoolin (None, 50, 6, 32)         0         
_________________________________________________________________
conv2d_183 (Conv2D)          (None, 50, 6, 64)         6208      
__________

Epoch 78/10000
 - 1s - loss: 6.7503e-05 - val_loss: 2.3518e-04
Epoch 79/10000
 - 1s - loss: 6.9155e-05 - val_loss: 2.3297e-04
Epoch 80/10000
 - 1s - loss: 1.0558e-04 - val_loss: 1.1557e-04
Epoch 81/10000
 - 1s - loss: 7.3907e-05 - val_loss: 1.7257e-04
Epoch 82/10000
 - 1s - loss: 6.2230e-05 - val_loss: 9.8071e-05
Epoch 83/10000
 - 1s - loss: 7.4150e-05 - val_loss: 1.2697e-04
Epoch 84/10000
 - 1s - loss: 5.2477e-05 - val_loss: 1.2469e-04
Epoch 85/10000


KeyboardInterrupt: 

# Case2

In [ ]:
X_data = X_data_Case12.copy()
X_data_test = X_data_test_Case12.copy()
Y_data = Y_data_Case12.copy()

In [ ]:
submit_case2_1 = run_model(X_data,X_data_test, Y_data, 4,False,False)
submit_case2_2 = run_model(X_data,X_data_test, Y_data, 5,False,False)
submit_case2_3 = run_model(X_data,X_data_test, Y_data, 6,False,False)

# Case 3

In [ ]:
X_data = X_data_Case3.copy()
X_data_test = X_data_test_Case3.copy()
Y_data = Y_data_Case3.copy()

In [ ]:
submit_case3_1 = run_model(X_data,X_data_test, Y_data, 7,False,True)
submit_case3_2 = run_model(X_data,X_data_test, Y_data, 8,False,True)
submit_case3_3 = run_model(X_data,X_data_test, Y_data, 9,False,True)

# Case 4

In [ ]:
X_data = X_data_Case4.copy()
X_data_test = X_data_test_Case4.copy()
Y_data = Y_data_Case4.copy()

In [ ]:
submit_case4_1 = run_model(X_data,X_data_test, Y_data, 10,True,False)
submit_case4_2 = run_model(X_data,X_data_test, Y_data, 11,True,False)
submit_case4_3 = run_model(X_data,X_data_test, Y_data, 12,True,False)

## 6. 결과 및 결언
## Conclusion & Discussion

# Make submission file

In [ ]:
Case5 = pd.read_csv("Case5.csv", index_col=0)

In [ ]:
Case1 = (submit_case1_1 + submit_case1_2 + submit_case1_3)/3
Case2 = (submit_case2_1 + submit_case2_2 + submit_case2_3)/3
Case3 = (submit_case3_1 + submit_case3_2 + submit_case3_3)/3
Case4 = (submit_case4_1 + submit_case4_2 + submit_case4_3)/3

In [ ]:
submission = ((Case1 + Case2)/2)*0.5 + ((Case3 + Case4 + Case5)/3)*0.5

In [ ]:
submission['X'] = np.clip(submission['X'], -400, 400)
submission['Y'] = np.clip(submission['Y'], -400, 400)
submission['M'] = np.clip(submission['M'],  25,  175)
submission['V'] = np.clip(submission['V'],  0.2, 1)

In [ ]:
submission.to_csv("submission_final.csv", index_col=0)